In [1]:
import numpy as np
import h5py
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

#IMPORT ALL PRE BUILD FUNCTION 
from func import *
# from sklearn.metrics import accuracy_score
import time
from momentum import *
from RMSprop import *
from adam import *

In [26]:
hf1 = h5py.File('/pythonfile/popular_optimizer/train.h5','r')  #READ THE DATASETS
X_train = hf1.get('X_train')     #GET THE X TRAINING DATASETS
Y_train = hf1.get('Y_train')

In [30]:
#flatten the array and normalize 
X_train = np.reshape(X_train,(X_train.shape[0],-1))  #FLATTEN   (209, 64 *64 * 3)
X_train = preprocessing.normalize(X_train)           #NORMALIZING THE DATASETS

Y_train = np.reshape(Y_train,(Y_train.shape[0],1))                # MAKING SURE IT IS IN CORRECT SHAPE

print("Shape of X_train {}  and  Y_train {}".format(X_train.shape,Y_train.shape))

Shape of X_train (259, 12288)  and  Y_train (259, 1)


In [37]:
def model(X,Y,learning_rate,num_iter,hidden_size,keep_prob,optimizer):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    
    if optimizer == 'Momentum':
        v = initilization_moment(params)

    elif optimizer == 'Rmsprop':
        s = initilization_RMS(params)

    elif optimizer == 'adam' :
        v,s = initilization_Adam(params)

    for i in range(1,num_iter):
        MiniBatches = RandomMiniBatches(X, Y, 64)
        p , q = MiniBatches[2]
        for MiniBatch in MiniBatches:

            (MiniBatch_X, MiniBatch_Y) = MiniBatch

            cache, A = model_forward(MiniBatch_X, params, L,keep_prob)
            cost = cost_f(A, MiniBatch_Y) 
            grad = backward(MiniBatch_X, MiniBatch_Y, params, cache, L,keep_prob)

            if optimizer == 'Momentum':
                params = update_params_with_momentum(params, grad, v, beta=0.9,learning_rate=learning_rate)

            elif optimizer == 'Rmsprop':
               params = update_params_with_RMS(params, grad, s, beta=0.9,learning_rate=learning_rate)

            elif optimizer == 'Adam' :
                params = update_params_with_Adam(params, grad,v, s, beta1=0.9,beta2=0.999,learning_rate=learning_rate,t=i)

           
        
        if i%100 == 0:
            print('cost of iteration______{}__________{}'.format(i,cost))
    return params

In [33]:
params = model(X_train, Y_train, learning_rate = 0.009,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='Adam')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration______500__________0.026226994301552467
train_accuracy------------ 0.9691119691119691


In [38]:
params = model(X_train, Y_train, learning_rate = 0.009,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='Momentum')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration______100__________0.6465357839100283
cost of iteration______200__________0.14667287714940405
cost of iteration______300__________0.5569837722631358
cost of iteration______400__________0.4387814209192945
cost of iteration______500__________0.2569025476023223
cost of iteration______600__________0.4815562414002521
cost of iteration______700__________0.4413088519845014
cost of iteration______800__________0.21030633214482936
cost of iteration______900__________0.14359594341871262
train_accuracy------------ 0.8996138996138996


In [39]:
params = model(X_train, Y_train, learning_rate = 0.009,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='RMSprop')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration______100__________0.7213054226936804
cost of iteration______200__________0.7288887939688129
cost of iteration______300__________0.6838691968163619
cost of iteration______400__________0.6796987230047025
cost of iteration______500__________0.6776842785109825
cost of iteration______600__________0.7221688378838073
cost of iteration______700__________0.685678944881559
cost of iteration______800__________0.7322927440436902
cost of iteration______900__________0.6919312289405452
train_accuracy------------ 0.4555984555984556


In [17]:
def model_simple_gradient_descent(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)

    for i in range(1,num_iter):

        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params(params, grad, learning_rate, L)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [18]:
params = model_simple_gradient_descent(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration________________ 20
0.6196480314939471
cost of iteration________________ 40
0.609287022014639
cost of iteration________________ 60
0.585299990570172
cost of iteration________________ 80
0.5867410369882348
cost of iteration________________ 100
0.5573366384458787
cost of iteration________________ 120
0.5481453600680758
cost of iteration________________ 140
0.540081161976517
cost of iteration________________ 160
0.5256660204617498
cost of iteration________________ 180
0.5357365574912608
train_accuracy------------ 0.8086124401913876


In [6]:
def model_momentum_optimizer(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    v = initilization_moment(params)
    for i in range(1,num_iter):
        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params_with_momentum(params, grad, v, beta=0.9,learning_rate=learning_rate)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [7]:
params = model_momentum_optimizer(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration________________ 20
0.6144856943253116
cost of iteration________________ 40
0.5715780411001979
cost of iteration________________ 60
0.5438768046959095
cost of iteration________________ 80
0.511577767817021
cost of iteration________________ 100
0.49070902690418217
cost of iteration________________ 120
0.47877285620702753
cost of iteration________________ 140
0.44634502661480824
cost of iteration________________ 160
0.42525388795193764
cost of iteration________________ 180
0.4168274516521236
train_accuracy------------ 0.8516746411483254


In [22]:
def model_RMSprop_optimizer(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    s = initilization_RMS(params)
    for i in range(1,num_iter):
        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params_with_RMS(params, grad, s, beta=0.9,learning_rate=learning_rate)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [23]:
params = model_RMSprop_optimizer(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration________________ 20
0.6426505857791115
cost of iteration________________ 40
0.6439747377347322
cost of iteration________________ 60
0.6419580033453186
cost of iteration________________ 80
0.6420333441950518
cost of iteration________________ 100
0.6421989031650565
cost of iteration________________ 120
0.6421855100610335
cost of iteration________________ 140
0.6422206926041393
cost of iteration________________ 160
0.6424180941588862
cost of iteration________________ 180
0.6421493894176203
train_accuracy------------ 0.6555023923444976


In [6]:
def model_Adam_optimizer(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    v,s = initilization_Adam(params)
    for i in range(1,num_iter):
        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params_with_Adam(params, grad,v, s, beta1=0.9,beta2=0.999,learning_rate=learning_rate,t=i)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [7]:
params = model_Adam_optimizer(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)

cost of iteration________________ 20
0.6917068244512788
cost of iteration________________ 40
0.6635906969014986
cost of iteration________________ 60
0.6446188405962355
cost of iteration________________ 80
0.6442435515110979
cost of iteration________________ 100
0.6440080034339223
cost of iteration________________ 120
0.6439739338702876
cost of iteration________________ 140
0.643974447999584
cost of iteration________________ 160
0.6439738156279745
cost of iteration________________ 180
0.6439737387058295


In [63]:
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

train_accuracy------------ 0.6555023923444976
